In [ ]:
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai_tools import DallETool


# ────────────────────────────────────────────────────────────────────────────────
# Variables - these can be changed:
# ────────────────────────────────────────────────────────────────────────────────

verbose_agents = True # A value of `True` (recommended) will instruct the Agent to output information while it is in progress; or `False` to get no information and just the result.


# ────────────────────────────────────────────────────────────────────────────────
# Initialize constants - please don't change these values:
# ────────────────────────────────────────────────────────────────────────────────

load_dotenv()

#scrape_website_tool = ScrapeWebsiteTool()

#geocoding_url = "https://geocoding-api.open-meteo.com/v1/search?name={{location}}"

#weather_url = "https://api.open-meteo.com/v1/forecast?latitude={{latitude}}&longitude={{longitude}}&daily=cloud_cover_mean,temperature_2m_mean,rain_sum,wind_speed_10m_mean&forecast_days=1&format=json&timeformat=unixtime"


# ────────────────────────────────────────────────────────────────────────────────
# Build the agents
# ────────────────────────────────────────────────────────────────────────────────
producer = Agent(
    role="Travel Agent Producer",
    goal="Produce, using your Crew, a written article about a location, with accompanying images.",
    backstory="""You are an expert producer and servant leader, skilled in gathering input from a human user and overseeing complex projects and guiding teams to success.
                 You can only ask for input from the human user. You cannot do any other task. You always must delegate other tasks to your crew.""",
    allow_delegation=True,
    verbose=verbose_agents,
    llm="openai/gpt-4.1",
)

user_surveyor = Agent(
    role="User Surveyor",
    goal="Ask questions to the human user, and return the response",
    backstory="You are an expert communicator. You ask for input from the human user and return their response as plain text.",
    verbose=verbose_agents,
)

geocoder = Agent(
    role="Geocoder",
    goal="Geocode a location in to a latitude and longitude",
    backstory="You have perfected the ability to geocode a location in to a latitude and longitude.",
    verbose=verbose_agents,
    llm="openai/gpt-4o",
)

weather_forecaster = Agent(
    role="Weather Forecaster",
    goal="Write a description about the weather",
    backstory="You are an expert Weather Forecaster who writes an engaging paragraph about the weather, very much like TV Weather Presenters, using well understood meteorological adjectives.",
    verbose=verbose_agents,
    llm="openai/o3-mini",
)

article_writer = Agent(
    role="Article Writer",
    goal="Craft written articles, that advertise and promote a location, its weather and its upcoming events",
    backstory="""You are an leading writer who writes for magazines, blogs and advertising.
                 You excel at gathering and presenting information clearly, concisely and in a friendly and polite manner.""",
    verbose=verbose_agents,
    llm="openai/gpt-4.1",
)

#geocoder = Agent(
#    role="Geocoder",
#    goal="Geocode a location in to a latitude and longitude, using a web API.",
#    backstory=f"""You have perfected the ability to geocode a location in to a latitude and longitude, from the specified Geocoding URL.
#                  You will need to pass the location in-place of the {{location}} placeholder in the specified Geocoding URL, before you call the URL.
#    
#                  Geocoding URL: {geocoding_url}
#
#                  From the results of calling the Geocoding URL, get the latitude and longitude from the first item in the United Kingdom.
#                  You are expected to return the result in JSON format.
#                  """,
#    tools=[scrape_website_tool],
#    verbose=verbose_agents,
#    llm="openai/o4-mini",
#)

#weather_forecaster = Agent(
#    role="Weather Forecaster",
#    goal="Write a description about the weather, using a web API",
#    backstory=f"""You are an expert Weather Forecaster who writes an engaging paragraph about the weather, very much like TV Weather Presenters,
#                  using well understood meteorological adjectives; and you can extract details about local weather from the specified web API.
#                  The details you can extract from the web API, and use in your weather forecasting and presentation, are the temperature, rain, wind speed and cloud cover.
#                 
#                  Web API: {weather_url}
#                    
#                  You will need to pass the actual latitude and longitude in-place of the {{latitude}} and {{longitude}}
#                  placeholders, respectively, in the specified web API URL, before you call the URL.
#                  """,
#    tools=[scrape_website_tool],
#    verbose=verbose_agents,
#    llm="openai/gpt-4o",
#)


# ────────────────────────────────────────────────────────────────────────────────
# Define the tasks
# ────────────────────────────────────────────────────────────────────────────────
test = Task(
    description="""Ask the user to input a location, either a town or city, in the United Kingdom;
                   and then write a short, engaging, fun and inviting article about local events and weather in that location.""",
    expected_output="A short article of plain text.",
    agent=producer,
    human_input=True,
)


# ────────────────────────────────────────────────────────────────────────────────
# Demo run
# ────────────────────────────────────────────────────────────────────────────────
result = Crew(
    agents=[
        user_surveyor,
        geocoder,
        weather_forecaster,
        article_writer,
    ],
    tasks=[
        test,
    ],
    process=Process.hierarchical,
    manager_agent=producer,
    memory=True,
    planning=True,
).kickoff()

print(result)
